<a href="https://colab.research.google.com/github/ModoSerio/Inteligencia_Artificial_Udea/blob/main/preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download vbookshelf/respiratory-sound-database

100% 3.68G/3.69G [00:35<00:00, 154MB/s]
100% 3.69G/3.69G [00:36<00:00, 110MB/s]


In [ ]:

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
import math
from matplotlib.cbook import boxplot_stats
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h3>Importando el diagnóstico </h3>

al mirar en el dataset el archivo llamado patient_diagnosis vemos que este tiene 2 columnas, en la cuál la primera es el ID del paciente y la segunda es la enfermedad que este tiene.


las abreviaciones usadas en la columna de diagnóstico son:
- COPD: Chronic Obstructive Pulmonary Disease
- LRTI: Lower Respiratory Tract Infection
- URTI: Upper Respiratory Tract Infection

In [ ]:
patient_data=pd.read_csv('/content/drive/MyDrive/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv', names=['pid','Enfermedad'])
patient_data.head()

,pid,Enfermedad
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


Ahora miremos como están organizados los datos en los archivos txt de los audios


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/103_2b2_Ar_mc_LittC2SE.txt',sep='\t')
df.head()

,0.364,3.25,0,1
0,3.250,6.636,0,0
1,6.636,11.179,0,1
2,11.179,14.250,0,1
3,14.250,16.993,0,1
4,16.993,19.979,0,0


según nos dice la descripción del dataset, las columnas significan lo siguiente:

Las cuatro columnas son:
- Comienzo de los ciclos respiratorios
- Fin del ciclo respiratorio
- Presencia/ausencia de crepitancias (presencia=1, ausencia=0)
- Presencia/ausencia de sibilancias (presencia=1, ausencia=0)

In [ ]:
path='/content/drive/MyDrive/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'
files=[s.split('.')[0] for s in os.listdir(path) if '.txt' in s]
files[:1]

['160_1b3_Al_mc_AKGC417L']

Cada audio está dividido en 5 elementos separados por un '_'. Estos 5 elementos son:

1. Número de paciente (101,102,...,226)
2. index de grabación
3. Ubicación
      a. Trachea (Tc)
      b. Anterior left (Al)
      c. Anterior right (Ar)
      d. Posterior left (Pl)
      e. Posterior right (Pr)
      f. Lateral left (Ll)
      g. Lateral right (Lr)
4. Modo de adquisición
     a. sequential/single channel (sc), 
     b. simultaneous/multichannel (mc)
5. Equipo de grabación
     a. AKG C417L Microphone (AKGC417L), 
     b. 3M Littmann Classic II SE Stethoscope (LittC2SE), 
     c. 3M Litmmann 3200 Electronic Stethoscope (Litt3200), 
     d.  WelchAllyn Meditron Master Elite Electronic  Stethoscope (Meditron)


Podemos convertir este formato a una lista para poder trabajar mejor de esta manera:

In [ ]:
def getFilenameInfo(file):
    return file.split('_')
getFilenameInfo('172_1b3_Ar_mc_AKGC417L')

['172', '1b3', 'Ar', 'mc', 'AKGC417L']

Ahora usando estos datos crearemos un dataset que resuma la información que está en patient diagnosis y en los archivos .txt

In [ ]:
files_data=[]
for file in files:
    data=pd.read_csv(path + file + '.txt',sep='\t',names=['inicio','fin','crepitancias','sibilancias'])
    name_data=getFilenameInfo(file)
    data['pid']=name_data[0]
    data['modo']=name_data[-2]
    data['nombre']=file
    files_data.append(data)
files_df=pd.concat(files_data)
files_df.reset_index()
files_df.head()

,inicio,fin,crepitancias,sibilancias,pid,modo,nombre
0,1.862,5.718,0,1,160,mc,160_1b3_Al_mc_AKGC417L
1,5.718,9.725,1,1,160,mc,160_1b3_Al_mc_AKGC417L
2,9.725,13.614,0,1,160,mc,160_1b3_Al_mc_AKGC417L
3,13.614,17.671,0,1,160,mc,160_1b3_Al_mc_AKGC417L
4,17.671,19.541,0,0,160,mc,160_1b3_Al_mc_AKGC417L


Por último mezclamos los dataframes del diagnóstico y el que acabamos de crear


In [ ]:
patient_data.pid=patient_data.pid.astype('int32')
files_df.pid=files_df.pid.astype('int32')
data=pd.merge(files_df,patient_data,on='pid')
data.head()

,inicio,fin,crepitancias,sibilancias,pid,modo,nombre,Enfermedad
0,1.862,5.718,0,1,160,mc,160_1b3_Al_mc_AKGC417L,COPD
1,5.718,9.725,1,1,160,mc,160_1b3_Al_mc_AKGC417L,COPD
2,9.725,13.614,0,1,160,mc,160_1b3_Al_mc_AKGC417L,COPD
3,13.614,17.671,0,1,160,mc,160_1b3_Al_mc_AKGC417L,COPD
4,17.671,19.541,0,0,160,mc,160_1b3_Al_mc_AKGC417L,COPD


<h3> Preprocesamiento de los audios

para preprocesar los audios, se creó un dataframe que contuviera los datos presentes en el archivo audio_and_txt_files. Además, se añadió una columna que tuviera la longitud del ciclo respiratorio, esto se hizo restando el tiempo final del ciclo con el tiempo de inicio

In [ ]:
root = path
def Extract_Annotation_Data(file_name, root):
    tokens = file_name.split('_')
    recording_annotations = pd.read_csv(os.path.join(root, file_name + '.txt'), names = ['inicio','fin','crepitancias','sibilancias'], delimiter= '\t')
    return (recording_annotations)

In [ ]:
rec_annotations_dict = {}
for s in files:
    a = Extract_Annotation_Data(s, root)
    rec_annotations_dict[s] = a
my_index = pd.Series(list(rec_annotations_dict.keys()))

In [ ]:
res_rate_list = []
for i in range (920):
    ciclo = pd.DataFrame( list(rec_annotations_dict.values())[i],columns=['inicio', 'fin' ,'Longitud de ciclo'])
    ciclo['Longitud de ciclo'] = ciclo['fin']-ciclo['inicio']
    prom = ciclo[["inicio","fin","Longitud de ciclo"]].mean(axis=0)
    ciclos_df= pd.DataFrame(prom).T
    res_rate_list.append(ciclos_df)

In [ ]:
res_rate_df = pd.concat(res_rate_list,ignore_index =True)
res_rate_df['filname']= my_index.values
res_rate_df['filname']= res_rate_df['filname'].str[:3]
res_rate_df.sort_values(by=['filname'])
res_rate_df.head(919)

,inicio,fin,Longitud de ciclo,filname
0,9.718000,13.253800,3.535800,160
1,9.718000,13.253800,3.535800,160
2,9.718000,13.253800,3.535800,160
3,8.769000,13.238750,4.469750,160
4,9.718000,13.253800,3.535800,160
...,...,...,...,...
105,9.986000,12.735167,2.749167,172
106,9.986000,12.735167,2.749167,172
107,9.986000,12.735167,2.749167,172
108,9.633714,12.242857,2.609143,172


<h3> Preprocesamiento de la información demográfica


In [ ]:
df_no_diagnosis = pd.read_csv('/content/drive/MyDrive/Respiratory_Sound_Database/demographic_info.txt', names = 
                 ['pid', 'Edad', 'Genero' , 'BMI (kg/m2)', 'Peso (kg)' , 'Child Height (cm)'],
                 delimiter = ' ')
df_no_diagnosis.head(10)

,pid,Edad,Genero,BMI (kg/m2),Peso (kg),Child Height (cm)
0,101,3.00,F,NaN,19.0,99.0
1,102,0.75,F,NaN,9.8,73.0
2,103,70.00,F,33.00,NaN,NaN
3,104,70.00,F,28.47,NaN,NaN
4,105,7.00,F,NaN,32.0,135.0
5,106,73.00,F,21.00,NaN,NaN
6,107,75.00,F,33.70,NaN,NaN
7,108,3.00,M,NaN,NaN,NaN
8,109,84.00,F,33.53,NaN,NaN
9,110,75.00,M,25.21,NaN,NaN


Mezclamos los dataframes de la informacion demografica  y el dataframe de diagnostico de pacientes

In [ ]:
df =  df_no_diagnosis.join(patient_data.set_index('pid'), on = 'pid', how = 'left')
df.head(10)


,pid,Edad,Genero,BMI (kg/m2),Peso (kg),Child Height (cm),Enfermedad
0,101,3.00,F,NaN,19.0,99.0,URTI
1,102,0.75,F,NaN,9.8,73.0,Healthy
2,103,70.00,F,33.00,NaN,NaN,Asthma
3,104,70.00,F,28.47,NaN,NaN,COPD
4,105,7.00,F,NaN,32.0,135.0,URTI
5,106,73.00,F,21.00,NaN,NaN,COPD
6,107,75.00,F,33.70,NaN,NaN,COPD
7,108,3.00,M,NaN,NaN,NaN,LRTI
8,109,84.00,F,33.53,NaN,NaN,COPD
9,110,75.00,M,25.21,NaN,NaN,COPD


Se remplazan los valores en NaN presentes en el dataframe y se eliminan las filas que consideramos no tendrían una información relevante 

In [ ]:
df.fillna(0, inplace=True)
df.head(10)

,pid,Edad,Genero,BMI (kg/m2),Enfermedad
0,101,3.00,F,0.00,URTI
1,102,0.75,F,0.00,Healthy
2,103,70.00,F,33.00,Asthma
3,104,70.00,F,28.47,COPD
4,105,7.00,F,0.00,URTI
5,106,73.00,F,21.00,COPD
6,107,75.00,F,33.70,COPD
7,108,3.00,M,0.00,LRTI
8,109,84.00,F,33.53,COPD
9,110,75.00,M,25.21,COPD


In [ ]:
print (df.shape)

(126, 5)
